In [1]:
import numpy as np
import torch # neural network library to test our implementation

import sys
import os

module_path = os.path.join(os.getcwd(), 'modules')
sys.path.append(module_path)
from mlp import MLP, CompoundNN
from activation_functions import ReLU, LogSoftmax
from losses import MSELoss, NLLLoss

#### Função para testar método forward em MLP

In [2]:
def test_forward_function():
    mlp = MLP(6, 5)
    x = np.random.randn(1, 6)
    out = mlp.forward(x)

    mlp_torch = torch.nn.Linear(6, 5) # Linear pois nossa MLP ainda não possui função de ativação
    mlp_torch.weight.data = torch.from_numpy(mlp.W).type(torch.float) # Copia pesos
    mlp_torch.bias.data = torch.from_numpy(mlp.b).type(torch.float) # Copia bias
    out_torch = mlp(torch.from_numpy(x).type(torch.float)) # Calcula saída e transforma para numpy
    out_torch = out_torch.data.numpy()

    error = ((out - out_torch) ** 2).mean() # Calcula erro médio quadrático
    if error < 1e-8:
        return 0
    else:
        return -1

#### Função para testar Compound Neural Network (Sequencia de MLP)

In [3]:
def test_compound_nn():
    mlp1 = MLP(6,5)
    relu1 = ReLU()
    mlp2 = MLP(5,4)
    relu2 = ReLU()

    nn = CompoundNN([mlp1, relu1, mlp2, relu2])

    x = np.random.randn(1, 6)

    out1 = relu2(mlp2(relu1(mlp1(x))))
    out2 = nn(x)

    error = ((out1 - out2)**2).mean()

    if error < 1e-8:
        return 0
    else:
        return -1

#### Função para testar save and load

In [4]:
def test_save_and_load():
    mlp1 = MLP(6,5)
    relu1 = ReLU()
    mlp2 = MLP(5,4)
    relu2 = ReLU()
    nn1 = CompoundNN([mlp1, relu1, mlp2, relu2])

    mlp1 = MLP(6,5)
    relu1 = ReLU()
    mlp2 = MLP(5,4)
    relu2 = ReLU()
    nn2 = CompoundNN([mlp1, relu1, mlp2, relu2])

    x = np.random.randn(1, 6)

    nn1.save('nn1')
    nn2.load('trainings/nn1')

    out1 = nn1(x)
    out2 = nn2(x)

    error = ((out1 - out2)**2).mean()

    if error < 1e-8:
        return 0
    else:
        return -1

#### Função para testar MSE Loss

In [5]:
def test_mse_loss():
    loss_fct = MSELoss()
    loss_fct(np.array([[1, 2]]), np.array([[1.4, 2.5]]))
    grad1 = loss_fct.backward()

    loss_fct = torch.nn.MSELoss()
    x = torch.tensor([[1., 2.]], requires_grad=True)
    loss = loss_fct(x, torch.tensor([[1.4, 2.5]]))
    loss.backward()
    grad2 = x.grad

    error = ((grad1 - grad2.data.numpy())**2).mean()

    if error < 1e-8:
        return 0
    else:
        return -1


#### Função para testar gradientes

In [6]:
def test_gradients():
    target = np.random.randn(1, 5)

    mlp = MLP(6, 5)
    x = np.random.randn(1, 6)
    out = mlp.forward(x)

    loss_fct = MSELoss()
    loss_fct(out, target)

    grad1 = loss_fct.backward()
    grad2 = mlp.backward(grad1)
    deltaw1 = mlp.deltaW


    mlp_torch = torch.nn.Linear(6, 5)
    mlp_torch.weight.data = torch.from_numpy(mlp.W).type(torch.float)
    mlp_torch.bias.data = torch.from_numpy(mlp.b).type(torch.float)
    out_torch = mlp_torch(torch.from_numpy(x).type(torch.float))
    out_torch = out_torch

    loss_fct = torch.nn.MSELoss()
    loss = loss_fct(out_torch, torch.from_numpy(target).type(torch.float))
    loss.backward()
    deltaw2 = mlp_torch.weight.grad.data.numpy()


    error = ((deltaw1.reshape(-1) - deltaw2.reshape(-1)) ** 2).mean()
    if error < 1e-8:
        return 0
    else:
        return -1

#### Função teste para LogSoftmax

In [7]:
def test_log_softmax():
    mlp1 = MLP(6,5)
    relu1 = ReLU()
    mlp2 = MLP(5,4)
    log_softmax = LogSoftmax()
    nn = CompoundNN([mlp1, relu1, mlp2, log_softmax])

    x = np.random.randn(1, 6)
    h = nn(x)
    log_probabilites = log_softmax(h)

    log_probabilites_torch = torch.nn.functional.log_softmax(torch.from_numpy(h).type(torch.float), dim=1)

    error = ((log_probabilites - log_probabilites_torch.numpy()) ** 2).mean()
    if error < 1e-8:
        return 0
    else:
        return -1

#### Função para testar gradiente jacobiana do logsoftmax

In [8]:
def test_log_softmax_gradient():
    x = np.random.randn(1, 6)
    log_softmax = LogSoftmax()
    log_probabilites = log_softmax(x)
    loss_fct = MSELoss()
    target = np.random.randn(1, log_probabilites.shape[1])
    loss_fct(log_probabilites, target)
    grad1 = loss_fct.backward()
    grad2 = log_softmax.backward(grad1)

    x_torch = torch.from_numpy(x).requires_grad_(True)
    log_probabilites_torch = torch.nn.functional.log_softmax(x_torch, dim=1)
    loss_fct = torch.nn.MSELoss()
    loss = loss_fct(log_probabilites_torch, torch.from_numpy(target))
    loss.backward()
    deltaw1 = x_torch.grad.data.numpy()
    
    error = ((grad2 - deltaw1) ** 2).mean()
    if error < 1e-8:
        return 0
    else:
        return -1

    

In [11]:
def test_log_softmax_gradient_nllloss():
    x = np.random.randn(1, 6)
    log_softmax = LogSoftmax()
    log_probabilites = log_softmax(x)
    loss_fct = NLLLoss()
    target = np.array([2])
    loss_fct(log_probabilites, target)
    grad1 = loss_fct.backward()
    grad2 = log_softmax.backward(grad1)

    x_torch = torch.from_numpy(x).requires_grad_(True)
    log_probabilites_torch = torch.nn.functional.log_softmax(x_torch, dim=1)
    loss_fct = torch.nn.NLLLoss()
    loss = loss_fct(log_probabilites_torch, torch.from_numpy(target))
    loss.backward()
    deltaw1 = x_torch.grad.data.numpy()
    
    error = ((grad2 - deltaw1) ** 2).mean()
    if error < 1e-8:
        return 0
    else:
        return -1

#### Assertions

In [12]:
assert test_compound_nn() == 0
assert test_forward_function() == 0
assert test_save_and_load() == 0
assert test_mse_loss() == 0
assert test_gradients() == 0
assert test_log_softmax() == 0
assert test_log_softmax_gradient() == 0
assert test_log_softmax_gradient_nllloss() == 0